# Algorithm Setup

In [1]:
import os
import shared
import networkx as nx
import numpy as np

import pyximport; pyximport.install()
import fennel

# ******** SECTION 1 ************* #

try:
    import config
except ImportError as err:
    print(err)
    print("**Could not load config.py\n**Copy config_template.py and rename it.")

pwd = os.getcwd()

DATA_FILENAME = os.path.join(pwd, "data", "oneshot_fennel_weights.txt")
OUTPUT_DIRECTORY = os.path.join(pwd, "output")

# Read input file for prediction model, if not provided a prediction
# model is made using FENNEL
PREDICTION_MODEL = ""

# File containing simulated arrivals. This is used in simulating nodes
# arriving at the shelter. Nodes represented by line number; value of
# 1 represents a node as arrived; value of 0 represents the node as not
# arrived or needing a shelter.
SIMULATED_ARRIVAL_FILE = os.path.join(pwd, "data", "simulated_arrival.txt")
#SIMULATED_ARRIVAL_FILE = ""

# File containing the geographic location of each node.
POPULATION_LOCATION_FILE = os.path.join(pwd, "data", "population_location.csv")

# Number of shelters
num_partitions = 4

# The number of iterations when making prediction model
num_iterations = 10

# Percentage of prediction model to use before discarding
# When set to 0, prediction model is discarded, useful for one-shot
prediction_model_cut_off = 0.10

# Alpha value used in one-shot (when restream_batches set to 1)
one_shot_alpha = 0.5

# Number of arrivals to batch before recalculating alpha and restreaming.
# When set to 1, one-shot is used with alpha value from above
restream_batches = 10

# Create virtual nodes based on prediction model
use_virtual_nodes = False

# Virtual nodes: edge weight
virtual_edge_weight = 1.0


####
# GRAPH MODIFICATION FUNCTIONS

# Also enables the edge calculation function.
graph_modification_functions = True

# If set, the node weight is set to 100 if the node arrives at the shelter,
# otherwise the node is removed from the graph.
alter_arrived_node_weight_to_100 = True

# Uses generalized additive models from R to generate prediction of nodes not
# arrived. This sets the node weight on unarrived nodes the the prediction
# given by a GAM.
# Needs POPULATION_LOCATION_FILE to be set.
alter_node_weight_to_gam_prediction = True

gam_k_value = 100

# Alter the edge weight for nodes that haven't arrived. This is a way to
# de-emphasise the prediction model for the unknown nodes.
prediction_model_emphasis = 1.0

SCOTCH Environment valid.
SCOTCH python bindings were loaded correctly from ../csap-graphpartitioning/src/python 
SCOTCH Library was located successfully at ../csap-graphpartitioning/tools/scotch/lib/macOS/libscotch.dylib


# Load Data

In [2]:
# read METIS file
G = shared.read_metis(DATA_FILENAME)

# Alpha value used in prediction model
prediction_model_alpha = G.number_of_edges() * (num_partitions / G.number_of_nodes()**2)

# Order of nodes arriving
arrival_order = list(range(0, G.number_of_nodes()))

# Arrival order should not be shuffled if using GAM to alter node weights
#random.shuffle(arrival_order)

if SIMULATED_ARRIVAL_FILE == "":
    # mark all nodes as needing a shelter
    simulated_arrival_list = [1]*G.number_of_nodes()
else:
    with open(SIMULATED_ARRIVAL_FILE, "r") as ar:
        simulated_arrival_list = [int(line.rstrip('\n')) for line in ar]
        
print("Graph loaded...")
print("Nodes: {}".format(G.number_of_nodes()))
print("Edges: {}".format(G.number_of_edges()))
if nx.is_directed(G):
    print("Graph is directed")
else:
    print("Graph is undirected")

Graph loaded...
Nodes: 1000
Edges: 2939
Graph is undirected


# Define Prediction Model

In [3]:
# setup for other algorithms
if config.ENABLE_SCOTCH == True:
    # import the relevant SCOTCH modules
    from scotch.graph_mapper import GraphMapper
    from scotch.io import ScotchGraphArrays

UNMAPPED = -1

# reset
assignments = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())
fixed = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())

print("PREDICTION MODEL")
print("----------------")

# Display which algorithm is being run
if config.PREDICTION_MODEL_ALGORITHM == config.Partitioners.FENNEL:
    print("Using: FENNEL Partitioning")
    print("---------------\n")
elif config.PREDICTION_MODEL_ALGORITHM == config.Partitioners.SCOTCH:
    print("Using: SCOTCH Partitioning")
    print("--------------------------\n")

predictionModels = {}
# store model data for different types of partitioners
# NOTE: THIS IS NOT IMPLEMENTED YET - need to discuss first
if config.RUN_ALL_PREDICTION_MODEL_ALGORITHMS == True:
    # create different prediction models
    fennelModel = {}
    fennelModel['assignments'] = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())
    fennelModel['fixed'] = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())

    predictionModels[config.Partitioners.FENNEL] = fennelModel

    scotchModel = {}
    scotchModel['assignments'] = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())
    scotchModel['fixed'] = np.repeat(np.int32(UNMAPPED), G.number_of_nodes())

    predictionModels[config.Partitioners.SCOTCH] = scotchModel

# Begin computation of prediction model
if PREDICTION_MODEL:
    # if we have a prediction model from file, load it
    with open(PREDICTION_MODEL, "r") as inf:
        assignments = np.fromiter(inf.readlines(), dtype=np.int32)

else:
    # choose the right algorithm
    if config.PREDICTION_MODEL_ALGORITHM == config.Partitioners.FENNEL:
        assignments = fennel.generate_prediction_model(G, num_iterations, num_partitions, assignments, fixed, prediction_model_alpha)
    elif config.PREDICTION_MODEL_ALGORITHM == config.Partitioners.SCOTCH:
        # SCOTCH algorithm
        # we have a networkx graph already, G
        scotchArrays = ScotchGraphArrays() # create the object storing all the SCOTCH arrays
        scotchArrays.fromNetworkxGraph(G, baseval=0) # populate arrays from G

        #scotchArrays.debugPrint() # uncomment this to print out contents of scotchArrays

        # create instance of SCOTCH Library
        mapper = GraphMapper(config.SCOTCH_LIB_PATH)

        # set some optional parameters for the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph
        # see csap-graphpartitioning/src/python/scotch/graph_mapper: GraphMapper.__init__() method for more options
        mapper.kbalval = 0.1
        mapper.numPartitions = num_partitions

        # intializes the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph using scotchArray and optional parameters
        ok = mapper.initialize(scotchArrays, verbose=False)
        if(ok):
            # we can proceed with graphMap, the data structures were setup correctly
            ok = mapper.graphMap()
            if(ok):
                # graphMap was run successfully, copy the assignments
                # make a deep copy as we then delete the mapper data, to clear memory
                # and the array reference may be lost
                assignments = np.array(mapper.scotchData._parttab, copy=True)

                mapper.delObjects()
            else:
                print('Error while running graphMap()')
        else:
            print('Error while setting up SCOTCH for partitioning.')

x = shared.score(G, assignments, num_partitions)
edges_cut, steps = shared.base_metrics(G, assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))

print("\nAssignments:")
shared.fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(G, assignments, num_partitions)

# save the prediction model
initial_model = np.array(assignments, copy=True)

PREDICTION MODEL
----------------
Using: SCOTCH Partitioning
--------------------------

WASTE		CUT RATIO	EDGES CUT	COMM VOLUME
0.05200		0.0214358625	63		107

Assignments:
[ 3  1  2  0  0  0  0  2  3  1  3  1  0  1  1  0  3  2  0  0  1  2  3  2  1  3  2  3  0  1  0  2  0  3  3  0  1  3  2  1  2  1  3  1  1  2  1  3  3  2  0  3  3  2  3  2  0  0  1  0  2  0  1  2  1  1  3  2  1  3  1  0  0  1  3  2  0  3  3  0  2  3  0  2  1  3  1  0  1  0  0  3  3  0  0  2  3  2  1  3  1  0  2  0  1  1  1  0  1  3  2  0  2  3  0  1  3  1  0  2  1  1  0  0  0  3  0  2  0  1  0  3  1  3  3  1  1  2  3  3  1  2  2  0  2  1  3  2  0  3  0  1  1  0  1  2  1  2  3  0  1  1  0  0  1  3  1  3  3  1  3  1  2  3  2  1  1  0  0  1  2  0  1  2  3  1  2  1  0  2  3  2  3  0  0  1  3  0  1  0  2  3  3  1  0  2  2  1  0  2  3  3  3  0  0  0  2  0  2  2  1  1  1  1  2  2  2  0  0  0  1  2  3  3  3  1  3  0  0  1  2  2  0  3  3  2  2  1  0  1  1  3  0  3  3  3  3  2  3  0  3  3  0  2  3  1  0  2  2  3  3  3  1  3  1  2

In [4]:
if False:
    G2 = nx.Graph()
    for node in G.nodes():
        G2.add_node(node)
        for neighbor in G.neighbors(node):
            G2.add_edge(node, neighbor)

    print(G2.number_of_nodes())
    print(G2.number_of_edges())



    # remove a random number of nodes (and respective edges)

    removed = [3,5,7,44,517,899, 989]

    for node in removed:
        for neighbor in G.neighbors(node):
            try:
                G2.remove_edge(node, neighbor)
            except Exception as err:
                try:
                    G2.remove_edge(neighbor, node)
                except Exception as err2:
                    print("Could not remove edge", node, neighbor, "as not found.")
        G2.remove_node(node)

    print(G2.number_of_nodes())
    print(G2.number_of_edges())


    # create a labels array
    G3 = nx.Graph()

    nodeIndex = {}
    nID = 0
    for node in G2.nodes():
        nodeIndex[node] = nID
        nID += 1

    for node in G2.nodes():
        G3.add_node(nodeIndex[node])
        for neighbor in G2.neighbors(node):
            G3.add_edge(nodeIndex[node], nodeIndex[neighbor])

    print(G3.number_of_nodes())
    print(G3.number_of_edges())

    assignmentsG3 = []

    # try partitioning this new graph
    scotchArrays = ScotchGraphArrays() # create the object storing all the SCOTCH arrays
    scotchArrays.fromNetworkxGraph(G3, baseval=0) # populate arrays from G

    #scotchArrays.debugPrint() # uncomment this to print out contents of scotchArrays

    # create instance of SCOTCH Library
    mapper = GraphMapper(config.SCOTCH_LIB_PATH)

    # set some optional parameters for the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph
    # see csap-graphpartitioning/src/python/scotch/graph_mapper: GraphMapper.__init__() method for more options
    mapper.kbalval = 0.1
    mapper.numPartitions = num_partitions

    # intializes the SCOTCH_Arch, SCOTCH_Strat, SCOTCH_Graph using scotchArray and optional parameters
    ok = mapper.initialize(scotchArrays, verbose=False)
    if(ok):
        # we can proceed with graphMap, the data structures were setup correctly
        ok = mapper.graphMap()
        if(ok):
            # graphMap was run successfully, copy the assignments
            # make a deep copy as we then delete the mapper data, to clear memory
            # and the array reference may be lost
            assignmentsG3 = np.array(mapper.scotchData._parttab, copy=True)
            print(assignmentsG3)
            #mapper.scotchData.debugPrint()
            mapper.delObjects()
        else:
            print('Error while running graphMap()')
    else:
        print('Error while setting up SCOTCH for partitioning.')
        #mapper.scotchData.debugPrint()




# Create Virtual Nodes

In [5]:
if use_virtual_nodes:
    print("Creating virtual nodes and assigning edges based on prediction model")

    # create virtual nodes
    virtual_nodes = list(range(G.number_of_nodes(), G.number_of_nodes() + num_partitions))
    print("\nVirtual nodes:")

    # create virtual edges
    virtual_edges = []
    for n in range(0, G.number_of_nodes()):
        virtual_edges += [(n, virtual_nodes[assignments[n]])]

    # extend assignments
    assignments = np.append(assignments, np.array(list(range(0, num_partitions)), dtype=np.int32))
    fixed = np.append(fixed, np.array([1] * num_partitions, dtype=np.int32))

    G.add_nodes_from(virtual_nodes, weight=1)
    G.add_edges_from(virtual_edges, weight=virtual_edge_weight)

    print("\nAssignments:")
    shared.fixed_width_print(assignments)
    print("Last {} nodes are virtual nodes.".format(num_partitions))

# Fix the first batch of arrivals

In [6]:
cut_off_value = int(prediction_model_cut_off * G.number_of_nodes())
if prediction_model_cut_off == 0:
    print("Discarding prediction model\n")
else:
    print("Assign first {} arrivals using prediction model, then discard\n".format(cut_off_value))

# fix arrivals
nodes_arrived = []
for a in arrival_order:
    # check if node needs a shelter
    if simulated_arrival_list[a] == 0:
        continue

    # set 100% node weight for those that need a shelter
    if alter_arrived_node_weight_to_100:
        G.node[a]['weight'] = 100

    nodes_fixed = len([o for o in fixed if o == 1])
    if nodes_fixed >= cut_off_value:
        break
    fixed[a] = 1
    nodes_arrived.append(a)

# remove nodes not fixed, ie. discard prediction model
for i in range(0, len(assignments)):
    if fixed[i] == -1:
        assignments[i] = -1

x = shared.score(G, assignments, num_partitions)
edges_cut, steps = shared.base_metrics(G, assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))

print("\nAssignments:")
shared.fixed_width_print(assignments)

nodes_fixed = len([o for o in fixed if o == 1])
print("\nFixed: {}".format(nodes_fixed))

shared.print_partitions(G, assignments, num_partitions)

Assign first 100 arrivals using prediction model, then discard

WASTE		CUT RATIO	EDGES CUT	COMM VOLUME
8.00000		0.1687648860	496		459

Assignments:
[-1  1  2  0 -1  0  0 -1 -1 -1  3 -1  0  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1  0 -1 -1  2 -1  3 -1 -1  1 -1 -1  1 -1  1 -1  1 -1  2  1 -1  3  2  0 -1 -1 -1 -1  2  0 -1  1  0 -1 -1 -1 -1  1 -1  3  2  1  3  1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  1 -1  1 -1 -1  0  0  3 -1  0  0 -1 -1 -1  1 -1  1 -1 -1 -1 -1  1 -1  0 -1  3 -1  0 -1 -1 -1  1  3  1 -1 -1  1 -1  0  0 -1  3 -1 -1  0  1  0 -1  1 -1 -1 -1  1 -1  3  3  1  2  2 -1 -1  1 -1  2 -1 -1 -1 -1  1  0  1  2  1 -1 -1 -1  1  1  0 -1  1  3  1  3 -1 -1 -1  1 -1  3 -1  1 -1 -1 -1  1 -1  0 -1 -1  3  1 -1  1 -1 -1 -1 -1 -1 -1  0  1 -1 -1  1  0 -1 -1 -1  1  0 -1 -1  1 -1 -1 -1 -1  3  0 -1  0 -1  0 -1 -1  1  1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

# Run Batch Arrival Mode

In [7]:
if restream_batches == 1:
    print("One-shot assignment mode")
    print("------------------------\n")
else:
    print("Assigning in batches of {}".format(restream_batches))
    print("--------------------------------\n")

    
# preserve original node/edge weight
if graph_modification_functions:
    node_weights = {n[0]: n[1]['weight'] for n in G.nodes_iter(data=True)}
    nx.set_node_attributes(G, 'weight_orig', node_weights)

    edge_weights = {(e[0], e[1]): e[2]['weight'] for e in G.edges_iter(data=True)}
    nx.set_edge_attributes(G, 'weight_orig', edge_weights)

Assigning in batches of 10
--------------------------------



In [10]:
# VERSION 3

G2 = nx.Graph()
for node in G.nodes():
    G2.add_node(node)

for node in G.nodes():
    for neighbor in G.neighbors(node):
        G2.add_edge(node, neighbor)

print(G2.number_of_nodes())
print(G2.number_of_edges())

scotch_assignments = [] # list of all the current assignments
scotch_arrived_nodes = [] # list of the nodes that have arrived
scotch_removed = []


import utilities.alg_utils as algutils

restream_batches = 10

pick_from_prediction = True # if both of these false, picks a random one
pick_from_minfill = True # overrides pick_from_prediction


finalG = None

current_batch = []
batchCount = 1
for i, a in enumerate(arrival_order):
    # TODO skip invalid nodes
    if graph_modification_functions:
        if simulated_arrival_list[a] == 0:
            # node does not arrive
            #G2.remove_node(a) # deletes all the edges related to this node
            scotch_removed.append(a)
            continue

        if alter_arrived_node_weight_to_100:
            #print('node_weight_altered')
            G.node[a]['weight'] = 100
    
    current_batch.append(a)
    
    # check if enough people have arrived to fill the batch or if this is the last batch
    if restream_batches == len(current_batch) or i == len(arrival_order) - 1:
        print('*******\nBatch #' + str(batchCount), 'arrived. Total nodes =', str(len(current_batch) + len(scotch_arrived_nodes)))

        '''
            Steps
            1. add new nodes and empty assignments from the batch
            2. add all the nodes in scotch_arrived_nodes to this empty graph
            3. add all the edges in scotch_arrived_nodes to the graph
        
        '''
        # step 1
        for newN in current_batch:
            scotch_arrived_nodes.append(newN)
            scotch_assignments.append(-1)
        
        # step 2
        G3 = nx.Graph()
        for node in scotch_arrived_nodes:
            G3.add_node(node)
        
        #print('G3.nodes', G3.nodes())
        
        # step 3: create all the edges that we can already create
        for node in G3.nodes():
            for neighbor in G.neighbors(node):
                # check neighbor is linked to a node in G3
                if neighbor != node and neighbor in scotch_arrived_nodes:
                    # add edge
                    G3.add_edge(node, neighbor)
                    
        #print('G3.baseedge', G3.edges())
        
        # determine which nodes are edgeless - create virtual edges
        virtualEdges = {}
        unassignedVirtualNodes = []
        
        nodes = G3.nodes()
        
        for node in nodes:
            if len(G3.neighbors(node)) == 0:
                # edgeless node
                # pick a partition
                partition = algutils.pickRandPartition(num_partitions)
                if pick_from_prediction:
                    partition = initial_model[node]

                if pick_from_minfill:
                    partition = algutils.minPartitionCounts(scotch_assignments, num_partitions)
                # pick another node in the same partition
                otherNode = algutils.getOtherNodeInPartition(node, partition, scotch_arrived_nodes, scotch_assignments)
                if otherNode is None:
                    # place in waiting list
                    unassignedVirtualNodes.append(node)
                else:
                    # create edge between these two nodes
                    uv = algutils.uvOrder(node, otherNode)
                    # store this edge
                    #algutils.addEdgeToIndex(virtualEdges, uv)
                    #print('Adding virt edge', uv)
                    if uv[0] in virtualEdges:
                        virtualEdges[uv[0]].append(uv[1])
                    else:
                        virtualEdges[uv[0]] = [uv[1]]
                    # add edge to graph
                    G3.add_edge(uv[0], uv[1])
                if node in current_batch:
                    #print('Edgeless unassigned', node)
                    # assign to the partition
                    scotch_assignments[algutils.getIndex(scotch_arrived_nodes, node)] = partition

        #print('assignments_updated', scotch_assignments)
        #print('virtual_edges', virtualEdges)
        #print('unaissingedNodes', unassignedVirtualNodes)
        #print('G3.nodes_updated',len(G3.nodes()), G3.nodes())
        #print('G3.edges_updated',len(G3.edges()), G3.edges())
        
        # checks
        for n in unassignedVirtualNodes:
            found = False
            for edge in G3.edges():
                if edge[0] == n or edge[1] == n:
                    found = True
            if(found == False):
                print("WARNING: unassigned Edge found?", n, found)
        
        
        
        
        # update node mapping for SCOTCH
        G4 = algutils.updateNodeMapping(G3, G)
        #print('G4',len(G4.nodes()), G4.nodes())
        #print('G4',len(G4.edges()), G4.edges())
        
        # update node weights and edge weights from G
        
        
        # partition
        scotchArrayData = ScotchGraphArrays()
        scotchArrayData.fromNetworkxGraph(G4, parttab=scotch_assignments, baseval=0)

        #print(scotchArrayData.velotab)

        scotchMapper = GraphMapper(config.SCOTCH_LIB_PATH, numPartitions=num_partitions)
        ok = scotchMapper.initialize(scotchArrayData, skipGraphValidStep=False, verbose=False)
        if(ok):
            # mapper initialized
            ok = scotchMapper.graphMapFixed()
            if(ok):
                scotch_assignments = scotchMapper.scotchData._parttab.tolist()
            else:
                print("Error running graphMapFixed()")
        else:
            print("Error initializing SCOTCH GraphMapper for graphMapFixed()")
        
        #print('assignments_after', scotch_assignments)
        
        finalG = G4
        # remove all virtual edges
        print('Removing', len(virtualEdges), 'virtual edges.')
        algutils.delVirtualEdges(finalG, virtualEdges)
        
        
        current_batch = []
        batchCount += 1
        #if len(scotch_arrived_nodes) == 3 * restream_batches:
        #    break
s_assignments = np.array(scotch_assignments)
x = shared.score(finalG, s_assignments, num_partitions)
edges_cut, steps = shared.base_metrics(finalG, s_assignments)
print("WASTE\t\tCUT RATIO\tEDGES CUT\tCOMM VOLUME")
print("{0:.5f}\t\t{1:.10f}\t{2}\t\t{3}".format(x[0], x[1], edges_cut, steps))        

print("\nAssignments:")
shared.fixed_width_print(s_assignments)

shared.print_partitions(finalG, s_assignments, num_partitions)

1000
2939
*******
Batch #1 arrived. Total nodes = 10
Removing 4 virtual edges.
*******
Batch #2 arrived. Total nodes = 20
Removing 11 virtual edges.
*******
Batch #3 arrived. Total nodes = 30
Removing 13 virtual edges.
*******
Batch #4 arrived. Total nodes = 40
Removing 19 virtual edges.
*******
Batch #5 arrived. Total nodes = 50
Removing 21 virtual edges.
*******
Batch #6 arrived. Total nodes = 60
Removing 23 virtual edges.
*******
Batch #7 arrived. Total nodes = 70
Removing 25 virtual edges.
*******
Batch #8 arrived. Total nodes = 80
Removing 27 virtual edges.
*******
Batch #9 arrived. Total nodes = 90
Removing 28 virtual edges.
*******
Batch #10 arrived. Total nodes = 100
Removing 27 virtual edges.
*******
Batch #11 arrived. Total nodes = 110
Removing 32 virtual edges.
*******
Batch #12 arrived. Total nodes = 120
Removing 28 virtual edges.
*******
Batch #13 arrived. Total nodes = 130
Removing 35 virtual edges.
*******
Batch #14 arrived. Total nodes = 140
Removing 33 virtual edges.
*

In [11]:
# Create a copy graph
G2 = nx.Graph()
for node in G.nodes():
    G2.add_node(node)

for node in G.nodes():
    for neighbor in G.neighbors(node):
        G2.add_edge(node, neighbor)

print(G2.number_of_nodes())
print(G2.number_of_edges())

scotch_assignments = [] # list of all the current assignments
scotch_arrived_nodes = [] # list of the nodes that have arrived
scotch_removed = []

restream_batches = 20


use_virtual_self_edges = False # True doesn't work...
replace_node_ids = True

current_batch = []
# iterate over all the arrivals
for i, a in enumerate(arrival_order):
    if graph_modification_functions:
        if simulated_arrival_list[a] == 0:
            # node does not arrive
            G2.remove_node(a) # deletes all the edges related to this node
            scotch_removed.append(a)
            continue

        if alter_arrived_node_weight_to_100:
            G2.node[a]['weight'] = 100

    current_batch.append(a)
    
    # check if enough people have arrived to fill the batch or if this is the last batch
    if restream_batches == len(current_batch) or i == len(arrival_order) - 1:
        print('Batch completed: ', current_batch)
        
        for newNode in current_batch:
            scotch_arrived_nodes.append(newNode)
            # add the non-fixed value ot scotch_assignments
            scotch_assignments.append(-1)
            
        print('arrived', scotch_arrived_nodes)

        G3 = nx.Graph()
        
        nodeNumOfNeighbors = {}
        
        # build a graph of ALL the arrived nodes
        for arrived in scotch_arrived_nodes:
            G3.add_node(arrived)
            
            # find neighbors of this node
            nodeNumOfNeighbors[arrived] = 0
            
            for neighbor in G2.neighbors(arrived):
                # check neighbor is also arrived
                if neighbor in scotch_arrived_nodes:
                    G3.add_edge(arrived, neighbor)
                    nodeNumOfNeighbors[arrived] += 1
            
        # list of fixed vertices that need a virtual edge
        virtual_edges = {}
        unassignedVirtual = {}
        # all the valid edges and nodes have been added to G3
        # create virtual edges
        for ii, nodeVal in enumerate(scotch_arrived_nodes):
            # for each node that has already arrived
            if nodeVal in current_batch:
                # node is a new node
                # compute number of neighbors
                # check the number of neighbors in the graph
                if len(G3.neighbors(nodeVal)) == 0:
                    #if nodeNumOfNeighbors[nodeVal] == 0:
                    # assign to a random partition
                    partition = 0
                    while True:
                        # generate a random number normally distributed * 4, with ceiling
                        partition = int(np.floor(abs(np.random.randn(1)[0]) * num_partitions))
                        if partition >= 0 and partition < num_partitions:
                            break
                    # fix this node
                    scotch_assignments[ii] = partition
                    
                    # find another node in this partition
                    found = False
                    for iii, otherPartition in enumerate(scotch_assignments):
                        if nodeVal == scotch_arrived_nodes[iii]:
                            continue # same edge
                        if partition != otherPartition:
                            continue # different partitions

                        # check u,v pair
                        u = nodeVal
                        v = scotch_arrived_nodes[iii]
                        if u > v:
                            tmp = u
                            u = v
                            v = tmp
                        # check if u,v pair already added
                        exists = False
                        for key in list(virtual_edges.keys()):
                            if key == u:
                                if v == virtual_edges[key]:
                                    exists = True
                                    break
                            else:
                                continue
                        if exists:
                            continue
                        
                        found = True
                        virtual_edges[u] = v
                        G3.add_edge(nodeVal, scotch_arrived_nodes[iii])
                        break
                    if found == False:
                        unassignedVirtual[nodeVal] = partition
        
        assignedVirtual = []
        for nodeVal in list(unassignedVirtual.keys()):
            for ii, otherPartition in enumerate(scotch_assignments):
                if unassignedVirtual[nodeVal] != otherPartition:
                    continue # wrong partition
                if nodeVal == scotch_arrived_nodes[ii]:
                    # skip, self edge
                    continue
                    
                u = nodeVal
                v = scotch_arrived_nodes[ii]
                if u > v:
                    tmp = u
                    u = v
                    v = tmp
                # check if u,v pair already added
                exists = False
                for key in list(virtual_edges.keys()):
                    if key == u:
                        if v == virtual_edges[key]:
                            exists = True
                            break
                    else:
                        continue
                if exists:
                    continue

                virtual_edges[u] = v
                G3.add_edge(nodeVal, scotch_arrived_nodes[ii])
                assignedVirtual.append(nodeVal)
                break
        
        for node in assignedVirtual:
            del unassignedVirtual[node]
            #unassignedVirtual.remove(node)
        
        if len(unassignedVirtual) != 0:
            print("Error, could not create virtual edges for each edgeless node:", unassignedVirtual)
                
        # perform partitioning
        print('assignments', scotch_assignments)

        scotchArrayData = ScotchGraphArrays()
        scotchArrayData.fromNetworkxGraph(G3, parttab=scotch_assignments, baseval=0, vlbltab=G3.nodes())

        print('virt_edge', virtual_edges)
        print('G3.edges', G3.edges())
        print('G3.nedge', G3.number_of_edges())
        print('edgesi', scotchArrayData.edgetab)
        print('edges', scotchArrayData._edgetab)
        print('nedges', len(scotchArrayData._edgetab))

        if replace_node_ids:
            # replace the array data with the scotch labels
            for newID, oldID in enumerate(scotch_arrived_nodes):
                print('arrivals', newID, oldID)
                # replace the oldID in edgetab with newID
                for i in range(0, len(scotchArrayData._edgetab)):
                    #for i, edgeval in enumerate(scotchArrayData._edgetab):
                    if scotchArrayData._edgetab[i] == oldID:
                        #print('adjust_arrival', i, edgeval)
                        scotchArrayData._edgetab[i] = newID

            print('adjusted_edges', scotchArrayData._edgetab)
            scotchArrayData._vlbltab = None            
        
        print(G3.number_of_nodes())
        print(G3.number_of_edges())
        
        scotchMapper = GraphMapper(config.SCOTCH_LIB_PATH, numPartitions=num_partitions)
        ok = scotchMapper.initialize(scotchArrayData, skipGraphValidStep=False)
        if(ok):
            # mapper initialized
            ok = scotchMapper.graphMapFixed()
            if(ok):
                scotch_assignments = scotchMapper.scotchData._parttab.tolist()
            else:
                print("Error running graphMapFixed()")
        else:
            print("Error initializing SCOTCH GraphMapper for graphMapFixed()")
            #scotchArrayData.debugPrint()
        '''
        # now, remove the temporary edges
        for key in list(tempEdges.keys()):
            try:
                scotchG.remove_edge(key, tempEdges[key])
            except nx.NetworkXError as err:
                # try other way round
                try:
                    scotchG.remove_edge(tempEdges[key], key)
                except nx.NetworkXError as err2:
                    pass
        '''
                
        print('assignments_after_fixing', scotch_assignments)
        current_batch = []
        break
                            
                


1000
2939
Batch completed:  [1, 2, 3, 5, 6, 10, 12, 13, 20, 24, 28, 31, 33, 36, 39, 41, 43, 45, 46, 48]
arrived [1, 2, 3, 5, 6, 10, 12, 13, 20, 24, 28, 31, 33, 36, 39, 41, 43, 45, 46, 48]
assignments [1, 0, 2, 3, 0, 2, 1, 2, 0, 1, 2, 1, 0, 1, 3, 1, 3, 2, 3, 2]
virt_edge {1: 12, 2: 6, 3: 10, 5: 39}
G3.edges [(1, 24), (1, 36), (1, 12), (1, 41), (1, 31), (2, 33), (2, 20), (2, 6), (3, 48), (3, 10), (3, 28), (3, 13), (3, 45), (5, 43), (5, 46), (5, 39)]
G3.nedge 16
edgesi [24, 36, 12, 41, 31, 33, 20, 6, 48, 10, 28, 13, 45, 1, 43, 46, 39, 2, 2, 1, 3, 5, 1, 3, 5, 3, 3, 5, 2, 1, 3, 1]
edges [24 36 12 41 31 33 20  6 48 10 28 13 45  1 43 46 39  2  2  1  3  5  1  3  5
  3  3  5  2  1  3  1]
nedges 32
arrivals 0 1
arrivals 1 2
arrivals 2 3
arrivals 3 5
arrivals 4 6
arrivals 5 10
arrivals 6 12
arrivals 7 13
arrivals 8 20
arrivals 9 24
arrivals 10 28
arrivals 11 31
arrivals 12 33
arrivals 13 36
arrivals 14 39
arrivals 15 41
arrivals 16 43
arrivals 17 45
arrivals 18 46
arrivals 19 48
adjusted_edges [ 

In [10]:
# SCOTCH VERSION

G2 = nx.Graph()
for node in G.nodes():
    G2.add_node(node)

for node in G.nodes():
    for neighbor in G.neighbors(node):
        G2.add_edge(node, neighbor)

scotchG = nx.Graph()
scotch_assignments = []
scotch_labels = []
scotch_removed = []

restream_batches = 25

current_batch = []
for i, a in enumerate(arrival_order):
    # check if node is already arrived
    if fixed[a] == 1:
        scotchG.add_node(a)
        scotch_labels.append(a)
        scotch_assignments.append(assignments[i])
        continue
    
    nodeW = 1
    
    # GRAPH MODIFICATION FUNCTIONS
    if graph_modification_functions:

        # remove nodes that don't need a shelter
        if simulated_arrival_list[a] == 0:
            G2.remove_node(a)
            scotch_removed.append(a)
            continue
        
        # set 100% node weight for those that need a shelter
        if alter_arrived_node_weight_to_100:
            G2.node[a]['weight'] = 100
            nodeW = 100

    current_batch.append(a)
    
    # check if enough people have arrived to fill the batch or if this is the last batch
    if restream_batches == len(current_batch) or i == len(arrival_order) - 1:
        print('********\nBatch completed:',len(current_batch), current_batch)
        
        # create a node for each new arrival
        # set the assignment to -1 for now
        for node in current_batch:
            #if(isinstance(scotch_assignments, np.array) == True):
            scotch_assignments.append(-1)
            #else:
            #    scotch_assignments.append(-1)
            scotch_labels.append(node)
            scotchG.add_node(node)

        tempEdges = {} # stores the list of edgeless nodes
        # now, add all the edges between nodes
        nodeID = 0
        nodes = scotchG.nodes()
        for node in nodes:
            #print('scotchGN - forstart', len(scotchG.nodes()))

            # get the neighbors for this node in the original graph, G
            neighbors = G2.neighbors(node)
            actual_neighbors = []
            for neighbor in neighbors:
                if neighbor in scotch_labels and node != neighbor:
                    scotchG.add_edge(node, neighbor)
                    actual_neighbors.append(neighbor)
                    
            #print('scotchGN - forneigh', len(scotchG.nodes()))

            if len(actual_neighbors) == 0:
                #print('Node has no neighbors', node, neighbors)
                # this node has no neighbors, currently, in this graph
                if node in current_batch:
                    # pick one of the partitions
                    partition = 0
                    while True:
                        # generate a random number normally distributed * 4, with ceiling
                        partition = int(np.floor(abs(np.random.randn(1)[0]) * num_partitions))
                        if partition >= 0 and partition < num_partitions:
                            break
                    
                    # fix this node to the new partition
                    scotch_assignments[ii] = partition

                    for i in range(0, len(scotch_assignments)):
                        if partition != scotch_assignments[i]:
                            continue
                        if nodes[i] == node:
                            continue
                        
                        scotchG.add_edge(node, nodes[i])
                    
                    ''' old code
                    partCounts = {}
                    partEdge = {}
                    assignmentID = 0
                    # count number of nodes in each partition
                    for partitionID in scotch_assignments:
                        if partitionID == -1:
                            continue
                        if partitionID not in partCounts:
                            partCounts[partitionID] = 1
                            partEdge[partitionID] = nodes[assignmentID]
                        else:
                            partCounts[partitionID] += 1
                        assignmentID += 1
                    
                    minVal = 100000
                    minPartitionID = -1
                    # find min partition
                    for partID in list(partCounts.keys()):
                        if partCounts[partID] < minVal:
                            minPartitionID = partID
                            minVal = partCounts[partID]
                    
                    # make an edge between node and a node in that partition
                    scotchG.add_edge(node, partEdge[minPartitionID])
                    tempEdges[node] = partEdge[minPartitionID]
                    #print('Found edge', node, partEdge[minPartitionID], minPartitionID)
                    '''
                    
                else:
                    # this is a node in one of the partitions
                    partitionID = scotch_assignments[nodeID]
                    for i in range(0, len(scotch_assignments)):
                        if i == nodeID:
                            continue
                        if scotch_assignments[i] != partitionID:
                            continue
                        # node at i is in the same partition as node at nodeID
                        scotchG.add_edge(node, nodes[i])
                        tempEdges[node] = nodes[i]
                        #print('Found edge:', node, nodes[i])
                        break
            nodeID += 1
        
        
        G3 = nx.Graph()

        nodeIndex = {}
        nID = 0
        for node in scotchG.nodes():
            nodeIndex[node] = nID
            nID += 1

        for node in scotchG.nodes():
            G3.add_node(nodeIndex[node])
            for neighbor in scotchG.neighbors(node):
                G3.add_edge(nodeIndex[node], nodeIndex[neighbor])

        print(scotchG.number_of_nodes())
        print(scotchG.number_of_edges())

        
        # run the partitioning code now
        scotchArrayData = ScotchGraphArrays()
        #scotchArrayData.fromNetworkxGraph(scotchG, parttab=scotch_assignments, baseval=0, vlbltab=scotch_labels)

        print('nassign', len(scotch_assignments))
        print('assignments_before', scotch_assignments)        
        
        scotchArrayData.fromNetworkxGraph(G3, parttab=scotch_assignments, baseval=0, vlbltab=scotch_labels)
        
        # replace the array data with the scotch labels
        for newID, oldID in enumerate(scotch_labels):
            # replace the oldID in edgetab with newID
            for i, edgeval in enumerate(scotchArrayData._edgetab):
                if edgeval == oldID:
                    scotchArrayData._edgetab[i] = newID

        scotchArrayData._vlbltab = None            
        
        print(G3.number_of_nodes())
        print(G3.number_of_edges())

        
        scotchMapper = GraphMapper(config.SCOTCH_LIB_PATH, numPartitions=num_partitions)
        ok = scotchMapper.initialize(scotchArrayData, skipGraphValidStep=False)
        if(ok):
            # mapper initialized
            ok = scotchMapper.graphMapFixed()
            if(ok):
                nl = []
                for item in scotchMapper.scotchData._parttab.tolist():
                    nl.append(item)
                #scotch_assignments = scotchMapper.scotchData._parttab.tolist()
                scotch_assignments = nl
            else:
                print("Error running graphMapFixed()")
        else:
            print("Error initializing SCOTCH GraphMapper for graphMapFixed()")
            #scotchArrayData.debugPrint()
        
        # now, remove the temporary edges
        for key in list(tempEdges.keys()):
            try:
                scotchG.remove_edge(key, tempEdges[key])
            except nx.NetworkXError as err:
                # try other way round
                try:
                    scotchG.remove_edge(tempEdges[key], key)
                except nx.NetworkXError as err2:
                    pass
                
        #break
        print('nassign', len(scotch_assignments))
        print('assignments_after', scotch_assignments)
        current_batch = []

    


********
Batch completed: 25 [223, 227, 230, 233, 234, 235, 236, 237, 239, 247, 248, 249, 251, 252, 254, 255, 260, 262, 265, 266, 267, 274, 277, 278, 280]
125
114
nassign 125
assignments_before [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
125
114
Intializing Architecture for GraphMap
   Architecture = True
Intializing Strategy for GraphMap
   Strategy = True
Loading Graph for GraphMap
   Graph = True
nassign 125
assignments_after [3, 0, 1, 1, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 0, 0, 3, 3, 3,

KeyboardInterrupt: 